<a href="https://colab.research.google.com/github/gl7176/CNN_tools/blob/main/Points_to_VIA_annotations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Connect to our Google Drive folder and pull files
Note: when you run this it will give you a link that you must click. You must give Google some permissions, then copy a code into a box that comes up in the output section of this code.

If customizing this code, you will need to point the `drive_folder` variable to a URL for your shared google drive folder.

In [1]:
# set variable to the destination google drive folder you want to pull from
drive_folder = 'https://drive.google.com/drive/folders/1Dav7-r1nxziYVE0ouKaG2xHzr-da9-WF'

# enter approximate seal length, in meters
seal_length = 2.6

!pip install -U -q PyDrive
import os, csv
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# choose a local (colab) directory to store the data.
local_download_path = os.path.expanduser('data')
try:
  os.makedirs(local_download_path)
except: pass

# 2. Auto-iterate using the query syntax
#    https://developers.google.com/drive/v2/web/search-parameters

# this bit points the code to that google drive folder
pointer = str("'" + drive_folder.split("/")[-1] + "'" + " in parents")

file_list = drive.ListFile(
    {'q': pointer}).GetList()

#    this bit pulls key files from the directory specified above
#    and checks that all necessary files are present


for f in file_list:
  # 3. Create & download by id.
  fname = os.path.join(local_download_path, f['title'])
  f_ = drive.CreateFile({'id': f['id']})
  f_.GetContentFile(fname)
  print("Pulled file: " + fname)
  if fname.endswith(".shp"):
    ptfile = fname
  if fname.endswith(".json"):
    tiling_scheme_file = fname

Pulled file: data/tiling_scheme (4).json
Pulled file: data/Jbn01_3_03_20_L1A_Seals.shp
Pulled file: data/Jbn01_3_03_20_L1A_Seals.cpg
Pulled file: data/Jbn01_3_03_20_L1A_Seals.dbf
Pulled file: data/Jbn01_3_03_20_L1A_Seals.sbx
Pulled file: data/Jbn01_3_03_20_L1A_Seals.shx
Pulled file: data/Jbn01_3_03_20_L1A_Seals.sbn
Pulled file: data/Jbn01_3_03_20_L1A_Seals.shp.xml
Pulled file: data/Jbn01_3_03_20_L1A_Seals.prj


In [2]:
!pip install geopandas
!pip install affine

     |████████████████████████████████| 1.0MB 7.7MB/s 
     |████████████████████████████████| 15.3MB 311kB/s 
     |████████████████████████████████| 6.5MB 40.3MB/s 


### Import tiling scheme information and set up variables and functions for coordinate conversions

In [106]:
import json

box_length = seal_length

with open(tiling_scheme_file) as f:
    tiling_scheme = json.load(f)
    tile_height = tiling_scheme["tile_height"]
    tile_width = tiling_scheme["tile_width"]
    tile_overlap = tiling_scheme["tile_overlap"]
    proj = tiling_scheme["spatial_reference"]
    gt = tiling_scheme["transform"]
    img_data = tiling_scheme["tile_pointers"]

from affine import Affine

geotransform = (gt[2], gt[0], gt[1], gt[5], gt[3], gt[4])
geotransform = Affine.from_gdal(*geotransform)

x_tile_dist = tile_width - tile_overlap
y_tile_dist = tile_height - tile_overlap

def locate_in_tile(bounding_box):
  entry = []
  lx_corner, lx_remainder = divmod(bounding_box[2][0], x_tile_dist)
  ty_corner, ty_remainder = divmod(bounding_box[2][1], y_tile_dist)
  rx_remainder = bounding_box[1][0] - (lx_corner*x_tile_dist)
  by_remainder = bounding_box[1][1] - (ty_corner*y_tile_dist)
  box_dimension = rx_remainder - lx_remainder
  tl_tile_point = "[{x}, {y}]".format(x=int(lx_corner*x_tile_dist), y=int(ty_corner*y_tile_dist))
  inv_map = {str(v): k for k, v in img_data['image_locations'].items()}
  if rx_remainder < tile_width+0.5*box_dimension and by_remainder < tile_height+0.5*box_dimension:
    tile_info = inv_map[tl_tile_point], int(lx_corner*x_tile_dist), int(ty_corner*y_tile_dist)
    entry.append(tile_info)
  #use remainder to determine whether a detection occurs in overlap and needs to be multiple-annotated
  disregard_threshold = 0.5
  # the above variable determines how "clipped" an edge box can be before we throw it away
  for i in [0,1]:
    if rx_remainder < tile_overlap+disregard_threshold*box_dimension:
      lx_corner = lx_corner-1
      tl_tile_point = "[{x}, {y}]".format(x=int(lx_corner*x_tile_dist), y=int(ty_corner*y_tile_dist))
      try:
        tile_info = inv_map[tl_tile_point], int(lx_corner*x_tile_dist), int(ty_corner*y_tile_dist)
        entry.append(tile_info)
      except:
        print("adjacent tile does not exist")
      #print("left margin")
      rx_remainder = rx_remainder+tile_overlap
    elif lx_remainder > tile_width-(tile_overlap+disregard_threshold*box_dimension):
      lx_corner = lx_corner+1
      tl_tile_point = "[{x}, {y}]".format(x=int(lx_corner*x_tile_dist), y=int(ty_corner*y_tile_dist))
      try:
        tile_info = inv_map[tl_tile_point], int(lx_corner*x_tile_dist), int(ty_corner*y_tile_dist)
        entry.append(tile_info)
      except:
        print("adjacent tile does not exist")
      #print("right margin")
      lx_remainder = lx_remainder-tile_overlap
    elif by_remainder < tile_overlap+disregard_threshold*box_dimension:
      ty_corner = ty_corner-1
      tl_tile_point = "[{x}, {y}]".format(x=int(lx_corner*x_tile_dist), y=int(ty_corner*y_tile_dist))
      try:
        tile_info = inv_map[tl_tile_point], int(lx_corner*x_tile_dist), int(ty_corner*y_tile_dist)
        entry.append(tile_info)
      except:
        print("adjacent tile does not exist")
      #print("top margin")
      by_remainder = by_remainder + tile_overlap
    elif ty_remainder > tile_height-(tile_overlap+disregard_threshold*box_dimension):
      ty_corner = ty_corner+1
      tl_tile_point = "[{x}, {y}]".format(x=int(lx_corner*x_tile_dist), y=int(ty_corner*y_tile_dist))
      try:
        tile_info = inv_map[tl_tile_point], int(lx_corner*x_tile_dist), int(ty_corner*y_tile_dist)
        entry.append(tile_info)
      except:
        print("adjacent tile does not exist")
      #print("bottom margin")
      ty_remainder = ty_remainder - tile_overlap
  return entry

### Convert boxes from UTM coordinates to tile-pixel coordinates

In [107]:
import fiona, ogr, numpy as np
from natsort import natsorted

box_list = []
with fiona.open(ptfile) as c:
    for i, record in enumerate(c):
        #import each point
        pt = ogr.Geometry(ogr.wkbPoint) 
        pt.SetPoint_2D(0, record["geometry"]["coordinates"][0], record["geometry"]["coordinates"][1])
        #buffer each point and draw "envelope" box around it
        buff = pt.Buffer(0.5*box_length)
        box = buff.GetEnvelope()
        bounding_box = [[box[0], box[2]], [box[1], box[2]], [box[0], box[3]], [box[1], box[3]]]        
        # transform box coordinates from global to orthomosaic
        for k, point in enumerate(bounding_box):
          #https://www.perrygeo.com/python-affine-transforms.html
          bounding_box[k] = ~geotransform * point    
        #transform box coodinates from orthomosaic to tile
        for k, q in enumerate(locate_in_tile(bounding_box)):
          tile_ID, x_offset, y_offset = locate_in_tile(bounding_box)[k]
          new_box = []
          for point in bounding_box:
            new_box.append((point[0]-x_offset, point[1]-y_offset))
            #print(k, bounding_box[k])
        # reassemble boxes entries with tile name and coordinates
          entry = {"tile_ID":tile_ID, "box": new_box}
          box_list.append(entry)
# sort boxes by tile name
def SortFunc(e):
  return e['tile_ID']
box_list = natsorted(box_list, key=SortFunc)

### Output detections in VIA format

In [108]:
# add class info later, when we have it on-hand to work with
# class_category = "Age Class"

#2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---28.png,1613979,"{}",4,0,"{""name"":""rect"",""x"":615,""y"":927,""width"":66,""height"":32}","{""Age Class"":""Adult""}"
#2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---28.png,1613979,"{}",4,1,"{""name"":""rect"",""x"":959,""y"":917,""width"":39,""height"":26}","{""Age Class"":""Pup""}"
new_line = [["filename","file_size","file_attributes","region_count","region_id","region_shape_attributes","region_attributes"]]

filename = ""
for detection in box_list:
    temp = []
    if filename != detection["tile_ID"]:
      filename = detection["tile_ID"]
      count = 0
    else:
      count += 1
    file_size = ""
    file_attributes = "{}"
    x1 = detection["box"][2][0]
    y1 = detection["box"][2][1]
    x2 = detection["box"][1][0]
    y2 = detection["box"][1][1]
    #print("x1={x1}, x2={x2}, y1={y1}, y2={y2}".format(x1=x1,x2=x2,y1=y1,y2=y2))
    region_shape_attributes = {"name":"rect", "x":x1, "y":y1, "width":x2-x1, "height":y2-y1}
    region_count = ""
    region_attributes = {}
    region_ID = count
    new_line.append([filename, file_size, file_attributes, region_count, region_ID, region_shape_attributes, region_attributes])

for k, x in enumerate(new_line):
  new_line[k][5],new_line[k][6] = str(x[5]).replace("'",'"'),str(x[6]).replace("'",'"')

In [109]:
# Set output directory, create it if necessary
output_dir = 'detection_outputs'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# write out new VIA file with additional detections
with open(output_dir + '/new_VIA_annotations.csv', 'w', newline='') as fp:
    writer = csv.writer(fp)
    writer.writerows(new_line)

from google.colab import files
files.download(output_dir + '/new_VIA_annotations.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>